In [1]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [11]:
import re
import pandas as pd
from lxml import etree
parser = etree.XMLParser(encoding='utf-8')
# Reading the XML file
docTree = etree.parse("pgs.jobxml", parser)

# Reading the configuration file to generate xpaths
df = pd.read_csv("xmldata.csv")

# initialize the underlying json
resultant_json = {"proc": {"summary": {"title": {"text": [], "append_to_log": []}}}}

In [33]:
xpath="/Parser/job/proc/summary/Param/Path[text()='::title:append to log']/.."
for i in docTree.xpath(xpath):
    #print(type(i.xpath("./Content")))
    if i.xpath("./Content"):
        #print(i.xpath("./Content"))
        print(i.xpath("./Content")[0].text)
    else:
        print(None)
    #print(i.xpath("./Content"))
#[book.get('author', None)  for book in tree.xpath('xpath')]
#[element.text if element.text else None for element in docTree.xpath(xpath)]

None
"value2"


In [14]:
# code2 
import re
import pandas as pd
from lxml import etree
parser = etree.XMLParser(encoding='utf-8')
# Reading the XML file
docTree = etree.parse("pgs.jobxml", parser)

import pandas as pd
df = pd.read_csv("xmldata.csv")
#df.groupby("File tag").filter(lambda gr: ~gr.File.str.contains("cream").any())
#print(df)
for i, j in df.groupby("parent"):
    #print(i)
    #print(j)
    #print("#")
    values={}
    for k, l in j.iterrows():
        #print(l)
        path_prefix = l['File node']
        tag = str(l["File tag"])
        #print(tag)
        xpath_constructor = ''
        xpath_constructor = "//" + "/".join(path_prefix.split(">")) + "/Param/Path[text()='{}']/..".format(tag)
        #print(tag)
        tag_value = [re.sub("\s", "_", items.strip("::")) for items in re.split("(?<!:):(?!:)", tag)][-1]
        #print(tag_value)
        values[tag_value]=[]
        for path in docTree.xpath(xpath_constructor):
            if path.xpath("./Content"):
                values[tag_value].append((path.xpath("./Content")[0].text))
            else:
                values[tag_value].append(None)
        #print(values)
        #print("#")
    print(values)
    combined={}
    #for q in combined:
        
    print("#")

{'text': [None, None], 'append_to_log': [None, '"value2"']}
#
{'auxiliary_headers_text_file': [None, '"value2 auxillary"']}
#


In [39]:
# to convert {'text': [None, None], 'append_to_log': [None, '"value2"']} 
# to [{'text': None, 'append_to_log': '"value2"'}]
a={'text': [None, None], 'append_to_log': [None, '"value2"']}
v=[]
first_value_len = len(list(a.values())[0])
for i in range(first_value_len):
    l={}
    #print(i)
    for qk, qv in a.items():
        #print(qk)
        l[qk] = qv[i]
    if not all(value == None for value in l.values()):
        v.append(l)

print(v)

[{'text': None, 'append_to_log': '"value2"'}]


In [37]:
l={'text': None, 'append_to_log': None}
all(value == None for value in l.values())
a={'text': [None, None], 'append_to_log': [None, '"value2"']}
list(a.values())[0]

[None, None]

In [ ]:
#resultant_json = {"proc": {"summary": {"title": {"text": [], "append_to_log": []}}}}

#resultant_json = {"proc": {"summary": {"title": [{"append_to_tag": "", "text": ""},]},
#                                      {"auxiliary_headers_text_file": []}}}


def get_item(resultant_json, path, value):
    for item in path[:-1]:
        resultant_json = resultant_json[item]
    resultant_json[path[-1]].extend(value)
    #print(value)
    #print(resultant_json)

for i, j in df.iterrows():
    # Iterate through each row of the configuration file,
    # Generate the xpath, pull the data from xml and dump the same to our json variable
    
    # path prefix - "proc > summary"
    path_prefix = j['File node']
    # tag - ::title:text
    tag = j["File tag"]

    if not "@" in tag:
        # A logic to not include Record header
        xpath_constrcutor = ''
        xpath_constrcutor = "//" + "/".join(path_prefix.split(">")) + "/Param/Path[text()='{}']/../Content".format(tag)
        
        # Constructed XPATH - //proc/summary/Param/Path[text()='::title:text']/../Content
        # get the value from the constructed xpath
        value = [element.text for element in docTree.xpath(xpath_constrcutor)]
        # get the json path to populate values there
        #tag_list = []
        #tag_list.extend(path_prefix.split(">"))
        #tag_list.extend([re.sub("\s", "_", items.strip("::")) for items in re.split("(?<!:):(?!:)", tag)])
        #tag_list = [element.strip() for element in tag_list]
        # get_item - method to populate the value to the json
        #get_item(resultant_json, tag_list, value)
        

print(resultant_json)
# resultant_json - {'proc': {'summary': {'title': {'text': [], 'append_to_log': ['"value1"', '"value2"']}}}}

In [ ]:

resultant_json = {"proc": {"summary": {"title": [{"append_to_tag": "", "text": ""},]},
                                      {"auxiliary_headers_text_file": []}}}


def get_item(resultant_json, path, value):
    for item in path[:-1]:
        resultant_json = resultant_json[item]
    resultant_json[path[-1]].extend(value)
    #print(value)
    #print(resultant_json)

for i, j in df.iterrows():
    # Iterate through each row of the configuration file,
    # Generate the xpath, pull the data from xml and dump the same to our json variable
    
    # path prefix - "proc > summary"
    path_prefix = j['File node']
    # tag - ::title:text
    tag = j["File tag"]

    if not "@" in tag:
        # A logic to not include Record header
        xpath_constrcutor = ''
        xpath_constrcutor = "//" + "/".join(path_prefix.split(">")) + "/Param/Path[text()='{}']/../Content".format(tag)
        # Constructed XPATH - //proc/summary/Param/Path[text()='::title:text']/../Content
        # get the value from the constructed xpath
        value = [element.text for element in docTree.xpath(xpath_constrcutor)]
        # get the json path to populate values there
        tag_list = []
        tag_list.extend(path_prefix.split(">"))
        tag_list.extend([re.sub("\s", "_", items.strip("::")) for items in re.split("(?<!:):(?!:)", tag)])
        tag_list = [element.strip() for element in tag_list]
        # get_item - method to populate the value to the json
        get_item(resultant_json, tag_list, value)
        

print(resultant_json)
# resultant_json - {'proc': {'summary': {'title': {'text': [], 'append_to_log': ['"value1"', '"value2"']}}}}